In [63]:
import pandas as pd
import nibabel as nib
from glob import glob
import numpy as np
import statsmodels.api as sp
#imported the full paths since it wasn't working otherwise - not sure why 
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.multitest import fdrcorrection


In [64]:
fc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_bold.pconn.nii'
sc_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-32k_atlas-schaefer_struc.pconn.nii'
sfc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii'


datasets=['HCP','LOBE']



In [66]:
sfc_paths={}
sfc_paths['HCP'] = sorted(glob(sfc_pattern.format(dataset='HCP',subject='*')))
sfc_paths['LOBE'] = sorted(glob(sfc_pattern.format(dataset='LOBE',subject='*')))

fc_paths={}
fc_paths['HCP'] = sorted(glob(fc_pattern.format(dataset='HCP',subject='*')))
fc_paths['LOBE'] = sorted(glob(fc_pattern.format(dataset='LOBE',subject='*')))

sc_paths={}
sc_paths['HCP'] = sorted(glob(sc_pattern.format(dataset='HCP',subject='*')))
sc_paths['LOBE'] = sorted(glob(sc_pattern.format(dataset='LOBE',subject='*')))

#print(sfc_paths)
#print(fc_paths)
#print(sc_paths)

In [ ]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_paths['LOBE'][0]).get_fdata().shape[1]

for dataset in datasets:
    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata()  for path in sfc_paths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_paths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_paths[dataset]] ,axis=0)

In [ ]:
##SFC T-test and FDR - LOBE vs HCP

In [ ]:
sfc['LOBE'].shape

In [28]:
x1 = sfc['HCP']
x2 = sfc['LOBE']

In [29]:
x2.shape

(14, 300)

In [30]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [31]:
pvals

array([1.70699709e-01, 1.78824864e-01, 2.26960136e-01, 1.88559300e-01,
       5.66890299e-01, 6.14631760e-01, 2.51952273e-01, 6.33617811e-01,
       4.13519648e-01, 7.38789719e-01, 8.39075179e-01, 1.03768428e-01,
       1.49555553e-01, 7.08002286e-01, 3.33001362e-01, 7.75473839e-01,
       2.85164578e-01, 2.14727308e-01, 5.78901077e-01, 5.89053450e-01,
       6.43934225e-02, 7.63964296e-02, 8.08575307e-03, 3.11774055e-04,
       8.87974178e-01, 7.75437313e-01, 9.24945788e-01, 2.60681315e-01,
       4.10744700e-01, 1.53628551e-01, 5.92039347e-01, 1.62312334e-01,
       8.60003899e-01, 4.21485002e-02, 4.13560234e-01, 8.06349544e-02,
       1.08980301e-02, 2.69804073e-01, 1.11270406e-01, 3.41460786e-02,
       6.03697259e-01, 5.46079259e-01, 4.87668753e-01, 1.27429067e-01,
       1.33117458e-01, 5.25288761e-01, 3.95303180e-01, 3.23160883e-01,
       3.27658719e-02, 1.02608151e-01, 3.16066632e-01, 5.67082663e-01,
       7.53478841e-01, 8.17526421e-01, 8.78900322e-01, 4.71310760e-01,
      

In [32]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [33]:
rejected

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [56]:
# FC avg per parcel, group difference
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [57]:
x1.shape

(341, 300, 300)

In [58]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [59]:
x1.shape

(341, 300)

In [60]:
x2.shape

(14, 300)

In [37]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [38]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [39]:
rejected

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [40]:
# SC avg per parcel, group difference
#set diagonal to be nan
x1 = sc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [41]:
x1.shape

(340, 300, 300)

In [42]:
x1 = np.nanmean(x1,axis=2)
x1.shape
x2 = np.nanmean(x2,axis=2)
x2.shape

(14, 300)

In [61]:
x1.shape

(341, 300)

In [62]:
x2.shape

(14, 300)

In [43]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [44]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [45]:
rejected

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,